In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pymysql
# from random import randint
import time
import redis

In [2]:
redi = redis.StrictRedis(host='localhost', port=6379,db=0)

In [3]:
def parsecontent(newsurl):
    url = newsurl
    try:
#     random_agent = USER_AGENTS[randint(0, len(USER_AGENTS)-1)]
        headers = {
            'Connection':'close',
        }
        r = requests.get(url,headers=headers)
        rt = r.content
        rh = str(rt,"utf-8")
        soup = bs(rh,"html.parser")
#     print(soup)

#     h1 = soup.find_all("h1")
#     print(h1)

#     "class":"arti-name"中包含部门、日期、浏览量
        info = soup.find_all("span",attrs={"class":"arti-name"})
#     print(info)
        infotext = info[0].get_text()
#     print(infotext)

#     爬取部门
#     +3为了除去“来源：”占用的3个字符,截至到20，20为年份开头
        department = infotext[infotext.find('来源：')+3:infotext.find('20')]
#     print(department)

#     正则匹配
        rn = re.compile('[\u4e00-\u9fa5]+')
        dep = rn.findall(department)
#     print(dep[0])
    
#     爬取时间
        date = infotext[infotext.find('20'):infotext.find('20')+10]
#     print(date)

#     由于浏览量的位数不确定，故不采用此方法
#     count = infotext[infotext.find('浏览量：')+4:infotext.find('浏览量：')+6]
#     print(count)

#     爬取浏览量
        countinfo = soup.find_all("span",attrs={"class":"WP_VisitCount"})
        count = countinfo[0].get_text()
#     print(count)

#     爬取内容
        contentinfo = soup.find_all("div",attrs={"class":"wp_articlecontent"})
        content = contentinfo[0].get_text()
#     print(content)

        return dep[0],date,count,content
    except:
        print("error:"+newsurl)
        return 0,0,0,0

In [4]:
def saveinfo(title,newsurl,department,date,count,content):
    db = pymysql.connect(host="localhost",user="zouzh",passwd="zouzihan0706",db="class")
    cursor = db.cursor()
    flag = redi.sadd('urls',newsurl)
    if flag == 1:
        try:
            cursor.execute("INSERT INTO cucnews2(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
            db.commit()
    #         print("db-ok")
        except:
    #         print("db-error")
            db.rollback()
    else:
        print("数据还未更新")
            
    db.close()

In [5]:
urlbegin = "http://www.cuc.edu.cn/news/1901/list"
# k用于计数
k = 0
try:
    for i in range(1,585):
#     random_agent = USER_AGENTS[randint(0, len(USER_AGENTS)-1)]
#     headers = {'User-Agent':random_agent,}
        url = urlbegin+str(i)+".htm"
        r = requests.get(url)
        rt = r.content
        rh = str(rt,"utf-8")
        soup = bs(rh,"html.parser")
        tit = soup.find_all("h3",attrs={'class','tit'})
        for t in tit:
            newsurl = t.find_all('a')
            urllen = str(newsurl[0]).find("target")
            title = t.get_text()
            newsurl = 'http://www.cuc.edu.cn' + (str(newsurl[0])[9:urllen-2])
#             对每个页面内容进行爬取
#             print(newsurl)
            department,date,count,content = parsecontent(newsurl)
#         测试
#         m = parsecontent(newsurl)
#         print(department)
            saveinfo(title,newsurl,department,date,count,content)
            k = k+1
            print(k)
            time.sleep(1)
    print("successful!")
except Exception as e:
    print(e)

Error 10061 connecting to localhost:6379. 由于目标计算机积极拒绝，无法连接。.
